In [2]:
import os
import openai
import pandas
from Bio import Entrez
import time
import json
import ast

In [ ]:
#juan
%env YOUR_ORG_ID=org-TpGuUg********************

%env OPENAI_API_KEY= sk-RDEji********************

openai.api_key = "sk-RDEji********************"

# list models
models = openai.Model.list()

model_select = "gpt-3.5-turbo" #OR gpt-4-0314

In [3]:
from pandas import read_csv

filename = "address-to-input-csv-file"

data = read_csv(filename)
print(data.shape)


(1360, 5)


In [4]:
data['joined_content']="NA"
data['Note']="NA"

data['Chat_complex_question']="NA"
data['token_complex_question']="NA"
data['FoundYES_complex_question']="NA"
data['FoundNO_complex_question']="NA"
data['FoundNOTSURE_complex_question']="NA"
data['FoundHOWEVER_complex_question']="NA"

In [5]:
outputpath='address-to-output-csv-file'

In [6]:
data.columns

Index(['rand_seed', 'selection', 'id', 'PMID', 'Title', 'joined_content',
       'Note', 'Chat_complex_question', 'token_complex_question',
       'FoundYES_complex_question', 'FoundNO_complex_question',
       'FoundNOTSURE_complex_question', 'FoundHOWEVER_complex_question'],
      dtype='object')

In [13]:
na_rows = data[data['FoundHOWEVER_complex_question']=='NA'].index.tolist()

In [14]:
print(na_rows)

[]


In [ ]:
for i in na_rows:
    try:
        print(i)
        temp_Title=data["Title"][i]
        temp_PMID=data["PMID"][i]
        #print(temp_PMID)
        #print(temp_Title)
        # Define the PMID of the article you want to retrieve
        pmid = temp_PMID

        # Use Entrez.efetch() to retrieve the article summary
        pmid_str = str(pmid)


        # Extract the article abstract from the record
        try:
            handle = Entrez.efetch(db='pubmed', id=pmid_str, retmode='xml')
            record = Entrez.read(handle)
            abstract = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
        except KeyError:
            abstract = 'No abstract found'

        temp_Abstract=" ".join(abstract)

        #print(temp_Abstract)
        joined_content=''.join(["Title: ",temp_Title,". Abstract: ",temp_Abstract])

        if len(joined_content) < 3000:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "qualified length"
        else:
            data["joined_content"][i] = joined_content
            data["Note"][i] = "length is too long"
            joined_content = joined_content[0:3000]

        print(joined_content)

        #complex_question#
        try:
            completion = openai.ChatCompletion.create(
                model=model_select, 
                messages=[{"role": "system", "content": "I want you to act as a helpful assistant. I will give you title and abstract of a publication and you will reply whether it meets all these following criteria or not: (1) studies that use human as primary research subject; (2) studies that involve patients with glioma, glioblastoma, astrocytoma, oligodendroglioma; (3) The title and abstract must explicitly mention that the study involves treatment of patients with one or more of the following immunotherapies: Immune Checkpoint Inhibitors (e.g. anti-PD-1, anti-PD-L1, anti-CTLA-4), Peptide Vaccination, Dendritic cell Vaccination, Adoptive Transfer of Effector Lymphocytes (e.g. CAR-T cell therapy, TCR-engineered T cells), Chimeric Antigen Receptor (CAR) T-cell therapy, or Oncolytic Viral Vectors therapy; (4) studies that was original research instead of reviews, protocol, or case report with less than five patients. Please answer whether it meets all these criteria or not with yes, no, or not sure, and followed with reasons."},
                          {"role": "user", "content": joined_content}]
            )
        except KeyError:
            completion = 'completion ERROR'
        #time.sleep(20)

        print(completion)
        if completion == 'completion ERROR':
            data["Chat_complex_question"][i]='completion ERROR'
            data["token_complex_question"][i]=0
        else:    
            data["Chat_complex_question"][i]=completion.choices[0].message['content']
            data["token_complex_question"][i]=completion.usage.total_tokens

        if "yes" in data["Chat_complex_question"][i][0:3].lower():
            data["FoundYES_complex_question"][i]="Found"
        else:
            data["FoundYES_complex_question"][i]="NotFound"

        if "no." in data["Chat_complex_question"][i][0:3].lower():
            data["FoundNO_complex_question"][i]="Found"
        elif "no," in data["Chat_complex_question"][i][0:3].lower():
            data["FoundNO_complex_question"][i]="Found"
        else:
            data["FoundNO_complex_question"][i]="NotFound"

        if "not sure" in data["Chat_complex_question"][i][0:10].lower():
            data["FoundNOTSURE_complex_question"][i]="Found"
        else:
            data["FoundNOTSURE_complex_question"][i]="NotFound"

        if "however" in data["Chat_complex_question"][i].lower():
            data["FoundHOWEVER_complex_question"][i]="Found"
        else:
            data["FoundHOWEVER_complex_question"][i]="NotFound"


        data.to_csv(outputpath,sep=',',index=True,header=True)
    except:
        pass